In [9]:
# Install your required packages here
!pip install pandas numpy matplotlib sklearn fsspec gcsfs

     |████████████████████████████████| 92kB 4.0MB/s 
     |████████████████████████████████| 1.2MB 6.1MB/s 
     |████████████████████████████████| 153kB 24.5MB/s 
     |████████████████████████████████| 266kB 31.7MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=f2fa090ec111ed0ccf80c86f027c9fe6df83c394f1ed91c5791d89bd8e2ca3be
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [2]:
import os

os.listdir()

['.config', 'ai-project-2020-f4dfbc25326c.json', 'sample_data']

In [1]:
%env GOOGLE_APPLICATION_CREDENTIALS=ai-project-2020-f4dfbc25326c.json

env: GOOGLE_APPLICATION_CREDENTIALS=ai-project-2020-f4dfbc25326c.json


In [2]:
%env GOOGLE_APPLICATION_CREDENTIALS=ai-project-2020-f4dfbc25326c.json

env: GOOGLE_APPLICATION_CREDENTIALS=ai-project-2020-f4dfbc25326c.json


In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout
from sklearn.preprocessing import LabelEncoder
from google.cloud import storage

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# define constants
bucket_name = "ai-project-2020-spotify"
client = storage.Client()
bucket = client.get_bucket(bucket_name)

In [6]:
train_files = list(bucket.list_blobs(prefix='training_set/'))
for blob in [blob for blob in train_files if '20180715' in blob.name]:
  print(blob.name)

training_set/log_0_20180715_000000000000.csv.gz
training_set/log_1_20180715_000000000000.csv.gz
training_set/log_2_20180715_000000000000.csv.gz
training_set/log_3_20180715_000000000000.csv.gz
training_set/log_4_20180715_000000000000.csv.gz
training_set/log_5_20180715_000000000000.csv.gz
training_set/log_6_20180715_000000000000.csv.gz
training_set/log_7_20180715_000000000000.csv.gz


In [11]:
logs = pd.read_csv(f"gs://{bucket_name}/training_set/log_0_20180715_000000000000.csv.gz")
logs.columns

Index(['session_id', 'session_position', 'session_length', 'track_id_clean',
       'skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch',
       'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
       'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
       'hour_of_day', 'date', 'premium', 'context_type',
       'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end'],
      dtype='object')

In [12]:
unique_tracks = logs['track_id_clean'].nunique()
print(unique_tracks)

319008


In [13]:
logs_dropped = logs[['session_id','session_position','track_id_clean', 'skip_2']]
# Skip_2 to binary
logs_dropped['skip_2'] = logs_dropped['skip_2'].astype(int)
logs_dropped.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,session_id,session_position,track_id_clean,skip_2
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,0
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,0
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,0
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,0
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5,t_64f3743c-f624-46bb-a579-0f3f9a07a123,0


In [14]:
encoder = LabelEncoder()
logs_dropped['track_id_clean'] = encoder.fit_transform(logs_dropped['track_id_clean'])
logs_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2990609 entries, 0 to 2990608
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   session_id        object
 1   session_position  int64 
 2   track_id_clean    int64 
 3   skip_2            int64 
dtypes: int64(3), object(1)
memory usage: 91.3+ MB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
def stack_sessions(df):
    """
    Turn matrix representation into vector by stacking the listen events together (as columns)
    For example:
    session_id session_position feature1 feature2
    a          1                ~        ~
    a          2                ~        ~
    b          1                ~        ~
    b          2                ~        ~
    b          3                ~        ~

    Turns into:
    session_id 1_feature1 1_feature2 2_feature1 2_feature2 3_feature1 3_feature2
    a          ~          ~          ~          ~          Nan        Nan
    b          ~          ~          ~          ~          ~          ~
    """
    columns = list(df.columns)
    columns.remove('session_id')
    columns.remove('session_position')
    sessions = df.pivot(index='session_id', columns='session_position', values=columns)
    return sessions

In [16]:
# Stack all Sessions
stacked_sessions = stack_sessions(logs_dropped)
# Drop all features except track_id, skip_2, session_ids
stacked_sessions.head()

track_id_clean            ... skip_2     
session_position                                   1         2   ...     19   20
session_id                                                       ...            
0_00006f66-33e5-4de7-a324-2d18e439fc1e         5621.0  180239.0  ...    1.0  1.0
0_0000a72b-09ac-412f-b452-9b9e79bded8f       290715.0    7754.0  ...    0.0  0.0
0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a       179103.0  303076.0  ...    1.0  1.0
0_00016a3d-9076-4f67-918f-f29e3ce160dc       140720.0  286600.0  ...    1.0  1.0
0_00018b58-deb8-4f98-ac5e-d7e01b346130        50130.0   58504.0  ...    NaN  NaN

[5 rows x 40 columns]

In [17]:
input_array = []
for index, row in stacked_sessions.iterrows():
    index = 3
    while index <= 8 :
        arr_row = {'Track1': row[('track_id_clean', index-2)], 'Track2': row[('track_id_clean', index-1)], 'Track3': row[('track_id_clean', index)], 'Track4': row[('track_id_clean', index+1)], 'Track5': row[('track_id_clean', index+2)], 'Track_3_Skip_2': row[('skip_2', index)]}
        input_array.append(arr_row)
        index = index+1
print(len(input_array))

1070052


In [18]:
model_input = pd.DataFrame(input_array)
model_input.head()

,Track1,Track2,Track3,Track4,Track5,Track_3_Skip_2
0,5621.0,180239.0,314424.0,44689.0,125848.0,0.0
1,180239.0,314424.0,44689.0,125848.0,249229.0,0.0
2,314424.0,44689.0,125848.0,249229.0,281749.0,0.0
3,44689.0,125848.0,249229.0,281749.0,14825.0,0.0
4,125848.0,249229.0,281749.0,14825.0,303773.0,1.0


In [19]:
model_input.shape


(1070052, 6)

In [20]:
skips = model_input['Track_3_Skip_2']

In [21]:
tracks = model_input.drop('Track_3_Skip_2', axis=1)

In [22]:
skips.shape

(1070052,)

In [23]:
tracks.shape

(1070052, 5)

In [24]:
embedding_size = 50
embedding = Embedding(input_dim=unique_tracks, output_dim=embedding_size, input_length=5, name='sequence_track_embedding')
model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(30, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(15, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
model.fit(x=tracks, y=skips, epochs=10, validation_split=0.2, batch_size=5000)

Epoch 1/10
172/172 [==============================] - 18s 104ms/step - loss: 0.6136 - accuracy: 0.6530 - val_loss: 0.7253 - val_accuracy: 0.5351
Epoch 2/10
172/172 [==============================] - 18s 103ms/step - loss: 0.5529 - accuracy: 0.6930 - val_loss: 0.8018 - val_accuracy: 0.5353
Epoch 3/10
172/172 [==============================] - 18s 102ms/step - loss: 0.4954 - accuracy: 0.7215 - val_loss: 0.9373 - val_accuracy: 0.5343
Epoch 4/10
172/172 [==============================] - 18s 103ms/step - loss: 0.4543 - accuracy: 0.7392 - val_loss: 1.1731 - val_accuracy: 0.5347
Epoch 5/10
172/172 [==============================] - 18s 103ms/step - loss: 0.4276 - accuracy: 0.7506 - val_loss: 1.4165 - val_accuracy: 0.5336
Epoch 6/10
172/172 [==============================] - 18s 102ms/step - loss: 0.4088 - accuracy: 0.7597 - val_loss: 1.7260 - val_accuracy: 0.5345
Epoch 7/10
172/172 [==============================] - 18s 103ms/step - loss: 0.3954 - accuracy: 0.7672 - val_loss: 1.9599 - val_ac

In [ ]:
model.save

In [32]:
embedding_layer = model.get_layer(name="sequence_track_embedding")
embedding_layer = pd.DataFrame(embedding_layer.get_weights()[0])
embedding_layer.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.033875,0.047370,0.023197,-0.020401,0.047857,-0.023679,0.036262,0.014599,0.014686,-0.041073,0.039454,0.036337,-0.031952,-0.039342,-0.011974,0.024340,-0.006942,0.033568,0.003854,-0.046212,0.010545,-0.030379,0.046308,0.032033,-0.032684,-0.011728,-0.046318,0.013488,0.031607,-0.022161,-0.015977,0.046706,0.022859,0.002011,0.014080,0.012164,-0.039286,0.035036,0.006925,0.027857,0.017201,-0.049906,-0.042533,-0.002784,-0.020299,0.001127,-0.037685,-0.018068,0.040775,0.036056
1,-0.036786,0.066691,-0.096675,0.058539,-0.004832,-0.049254,-0.018054,-0.002008,-0.047270,-0.031952,-0.053679,-0.040039,-0.094667,0.121376,-0.071154,0.039361,-0.005061,0.115382,0.110668,0.024943,-0.114266,-0.003926,0.007115,0.060059,-0.009180,0.124521,0.048334,-0.033978,-0.012116,0.054881,-0.044650,-0.008491,0.008106,-0.009795,0.126547,-0.020160,0.044465,-0.023339,0.055788,-0.092280,-0.098802,-0.020120,0.070625,-0.018232,0.062357,0.004222,-0.007548,-0.045778,0.073257,-0.082723
2,-0.028315,0.044363,-0.059785,0.111413,-0.014017,-0.161652,-0.012663,-0.005749,-0.002311,-0.226310,-0.030573,0.042760,-0.131752,0.057019,0.030567,0.075118,0.166251,-0.013411,0.013798,-0.011878,-0.091362,0.014154,0.103645,0.010505,-0.151111,0.059803,-0.136762,-0.039064,-0.137698,0.129964,-0.046579,-0.060361,0.041188,-0.141306,0.028044,-0.075520,0.066755,0.052113,-0.008015,0.111465,-0.012977,-0.105865,0.111954,-0.116224,-0.055370,-0.020840,0.081447,-0.048186,-0.029952,-0.024110
3,0.026977,0.000909,-0.030441,0.030630,-0.019343,-0.015275,-0.006694,-0.040566,-0.028212,-0.030661,-0.020765,0.049071,-0.014045,-0.004874,-0.030092,0.046760,0.006824,-0.027269,0.049418,-0.048481,0.034839,0.012267,0.038519,0.045025,0.026352,-0.045968,0.028785,0.004690,0.019502,0.001590,-0.028143,-0.026611,0.020121,0.001280,-0.027488,-0.002379,-0.023885,-0.037641,0.025096,-0.047240,0.014771,0.027699,-0.021025,0.044378,0.026376,0.021001,-0.013994,-0.010268,-0.028588,-0.039348
4,0.005791,-0.056246,-0.044965,-0.022959,-0.019970,-0.053835,0.030764,0.020124,-0.088946,0.014746,0.083345,-0.027527,-0.071402,-0.031327,0.034442,0.022711,0.010025,-0.043805,0.125235,-0.021676,0.085293,-0.041753,0.009159,-0.100616,0.068117,-0.018142,-0.008547,-0.016521,-0.031362,0.043372,-0.032608,-0.028152,-0.013695,-0.012240,-0.046511,-0.004607,0.011858,0.014594,-0.087158,-0.046952,0.029584,-0.014815,0.022486,0.000603,0.055311,0.021881,-0.020985,0.058969,-0.003666,-0.005771


In [33]:
embedding_layer.to_csv('sequence_track_embedding_weights.csv')

In [34]:
embedding_layer.shape

(319008, 50)

In [ ]:
embedding_layer.index = encoder.inverse_transform(embedding_layer.index)

In [ ]:
!git clone https://github.com/chiru-stefan/AE-Emb---AI-Proj.git